# import

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from selenium import webdriver
from selenium.webdriver import ActionChains

import time
import datetime
import random

from lxml import etree
from selenium.webdriver.common.keys import Keys
import json

# Function

## Umeng Functions

In [2]:
def open_window():
    '''
    打开一个chrome窗口，停用开发者模式
    实例化浏览器对象
    return 一个driver对象
    '''
    chrome_options = webdriver.ChromeOptions()

    # 设置浏览器初始 位置x,y & 宽高x,y
    chrome_options.add_argument(f'--window-position={217},{172}')
    chrome_options.add_argument(f'--window-size={1200},{1000}')

    # 关闭自动测试状态显示 // 会导致浏览器报：请停用开发者模式
    # window.navigator.webdriver还是返回True,当返回undefined时应该才可行。
    chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])
    prefs = {
            'profile.default_content_setting_values':
                {
                    'notifications': 2
                }
        }
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_experimental_option("useAutomationExtension", False)
    #chrome_options.add_argument('--headless') 
    #隐藏浏览器
    # 部署项目在linux时，其驱动会要求这个参数
    #chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument(
            'user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"')
    #实例化浏览器对象
    driver = webdriver.Chrome(executable_path = './chromedriver', options = chrome_options)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})""",
        })
    return driver

In [3]:
def get_cookie_dic(s):
    '''
    将cookie元数据输入，返还一个cookie的dictionary，用于登陆网站以及post/get命令请求
    '''
    dic_c = {}
    for x in s.split('; '):
        dic_c[x.split("=")[0]] = x.split("=")[1]
    return dic_c

In [4]:
def get_total_trend(driver, dic_c , date_begin, date_end, platform):
    '''
    get total trend information from Umeng and export data into total_trend.csv
    '''
    driver.get("https://mobile.umeng.com/platform/{}/reports/trend_summary".format(platform))
    time.sleep(1)
    
    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    
    time.sleep(2)
    
    date1 = driver.find_elements_by_class_name("ant-calendar-range-picker-input")
    date1[0].click()

    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[0])
    time.sleep(2)
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(date_end[1])
    
    time.sleep(2)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    
    button = driver.find_element_by_xpath('//*[@id="scroll-container"]/div/div[2]/div/div/div[3]/div[4]/div[1]/div[1]/span/div[2]/a/span')
    button.click()

    time.sleep(2)
    
    head_list = driver.find_elements_by_class_name("ant-table-column-title")
    header_list = [i.text for i in head_list]
    total_trend = pd.DataFrame( columns= header_list)
    time.sleep(5)
    data_list = driver.find_elements_by_class_name("ant-table-row")
    for i in data_list:
        total_trend.loc[len(total_trend)] = i.text.split(" ")
    
    total_trend.to_csv("C:\\Users\\Ker\\Desktop\\total_trend.csv",encoding = "utf-8_sig")

In [5]:
def get_user_stay(driver, dic_c, date_begin, date_end, platform):
    '''
    return 两周的新用户存留表到new_user_stay.csv, 活跃用户留存表到active_user_stay.csv
    '''
    driver.get("https://mobile.umeng.com/platform/{}/reports/retention".format(platform))
    time.sleep(1)
    
    print("am i here?")
    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    print("am i here????")
    button = driver.find_element_by_xpath('//*[@id="scroll-container"]/div/div[2]/div/div[2]/div[1]/div[1]/div[1]/span/div[1]/div/label[1]/span[2]/span')
    button.click()
    time.sleep(1)
    
    date1 = driver.find_elements_by_class_name("ant-calendar-range-picker-input")
    date1[0].click()

    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[0])
    time.sleep(2)
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(date_end[1])
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    
    head_list = driver.find_elements_by_class_name("ant-table-column-title")
    count = len(head_list)
    header_list = [i.text for i in head_list]
    #print(header_list)

    new_user_stay = pd.DataFrame( columns= header_list)
    time.sleep(5)
    data_list = driver.find_elements_by_class_name("ant-table-row")
    for i in data_list:
        l  = i.text.split("\n")
        if len(l) < count:
            for j in range(count-len(l)):
                l.append("")
        new_user_stay.loc[len(new_user_stay)] = l
        
    new_user_stay.to_csv("C:\\Users\\Ker\\Desktop\\new_user_stay.csv",encoding = "utf-8_sig")
    
    #活跃用户
    button = driver.find_element_by_xpath('//*[@id="scroll-container"]/div/div[2]/div/div[2]/div[1]/div[1]/div[1]/span/div[1]/div/label[2]/span[2]/span')
    button.click()
    time.sleep(1)
    
    head_list = driver.find_elements_by_class_name("ant-table-column-title")
    count = len(head_list)
    header_list = [i.text for i in head_list]
    #print(header_list)

    active_user_stay = pd.DataFrame( columns= header_list)
    time.sleep(5)
    data_list = driver.find_elements_by_class_name("ant-table-row")
    for i in data_list:
        l  = i.text.split("\n")
        if len(l) < count:
            for j in range(count-len(l)):
                l.append("")
        active_user_stay.loc[len(active_user_stay)] = l
        
    active_user_stay.to_csv("C:\\Users\\Ker\\Desktop\\active_user_stay.csv",encoding = "utf-8_sig")

In [6]:
def get_comma_out(x):
    if ',' in str(x):
        return str(x).replace(',','')
    else:
        return x

In [7]:
def get_device(driver, dic_c, date_begin, date_end, platform):
    '''
    return 两周的设备终端信息到 device.csv
    '''
    driver.get("https://mobile.umeng.com/platform/{}/reports/device".format(platform))
    time.sleep(1)
    
    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    date = driver.find_element_by_class_name("ant-calendar-picker-icon")
    date.click()
    
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    head = driver.find_elements_by_class_name("ant-table-column-title")[4:]
    header_list = [i.text for i in head]
    device1 = pd.DataFrame( columns= header_list)
    
    
    while driver.find_element_by_class_name("ant-pagination-next").get_attribute("aria-disabled") == 'false':
        data = driver.find_elements_by_class_name("ant-table-row")[4:]
        time.sleep(1)
        for i in data:
            l = i.text.split(' ')
            if len(l) > len(header_list):
                result = []
                phone = ' '.join(l[:len(l)-len(header_list)+1])
                result.append(phone)
                result.extend(l[len(l)-len(header_list)+1:])
                device1.loc[len(device1)] = result
            else:
                device1.loc[len(device1)] = l
        driver.find_element_by_class_name("ant-pagination-next").click()
        time.sleep(1)
    
    data = driver.find_elements_by_class_name("ant-table-row")[4:]
    for i in data:
        l = i.text.split(' ')
        if len(l) > len(header_list):
            result = []
            phone = ' '.join(l[:len(l)-len(header_list)+1])
            result.append(phone)
            result.extend(l[len(l)-len(header_list)+1:])
            device1.loc[len(device1)] = result
        else:
            device1.loc[len(device1)] = l
    
    #switch date
    date = driver.find_element_by_class_name("ant-calendar-picker-icon")
    date.click()
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_end[1])
    time.sleep(1)
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    device2 = pd.DataFrame( columns= header_list)
    
    
    while driver.find_element_by_class_name("ant-pagination-next").get_attribute("aria-disabled") == 'false':
        data = driver.find_elements_by_class_name("ant-table-row")[4:]
        time.sleep(1)
        for i in data:
            l = i.text.split(' ')
            if len(l) > len(header_list):
                result = []
                phone = ' '.join(l[:len(l)-len(header_list)+1])
                result.append(phone)
                result.extend(l[len(l)-len(header_list)+1:])
                device2.loc[len(device2)] = result
            else:
                device2.loc[len(device2)] = l

        driver.find_element_by_class_name("ant-pagination-next").click()
        time.sleep(1)
        
    data = driver.find_elements_by_class_name("ant-table-row")[4:]
    for i in data:
        l = i.text.split(' ')
        if len(l) > len(header_list):
            result = []
            phone = ' '.join(l[:len(l)-len(header_list)+1])
            result.append(phone)
            result.extend(l[len(l)-len(header_list)+1:])
            device2.loc[len(device1)] = result
        else:
            device2.loc[len(device1)] = l
            
    device1 = device1[['机型','新增用户','启动次数']]
    device2 = device2[['机型','新增用户','启动次数']]
    device =  pd.merge(device1, device2, on='机型',how = 'outer')
    
    device = device.fillna(0)
    device['启动次数_x'] = device['启动次数_x'].apply(get_comma_out)
    device['启动次数_y'] = device['启动次数_y'].apply(get_comma_out)
    device['新增用户_x'] = device['新增用户_x'].apply(get_comma_out)
    device['新增用户_y'] = device['新增用户_y'].apply(get_comma_out)
    
    device['新增用户_x'] = device['新增用户_x'].astype(int)
    device['新增用户_y'] = device['新增用户_y'].astype(int)
    device['启动次数_x'] = device['启动次数_x'].astype(int)
    device['启动次数_y'] = device['启动次数_y'].astype(int)

    device['新增用户'] = device["新增用户_x"]+device["新增用户_y"]
    device['启动次数'] = device["启动次数_x"]+device["启动次数_y"]
    device = device[['机型','新增用户','启动次数']]
    
    device.to_csv("C:\\Users\\Ker\\Desktop\\device.csv",encoding = "utf-8_sig")

In [8]:
def get_location(driver, dic_c, date_begin, date_end, platform):
    '''
    return location information into province.csv and country.csv
    '''
    driver.get("https://mobile.umeng.com/platform/{}/reports/location".format(platform))
    time.sleep(1)
    
    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    button = driver.find_element_by_xpath('//*[@id="scroll-container"]/div/div[2]/div/div[1]/div[2]/div[1]/div/label[1]/span[2]/span')
    button.click()
    time.sleep(1)
    
    date = driver.find_element_by_class_name("ant-calendar-picker-icon")
    date.click()
    time.sleep(1)
    
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    head = driver.find_elements_by_class_name("ant-table-column-title")
    header_list = [i.text for i in head]
    header_list
    
    province = pd.DataFrame( columns= header_list)
    
    
  
    while driver.find_element_by_class_name("ant-pagination-next").get_attribute("aria-disabled") == 'false':
        data = driver.find_elements_by_class_name("ant-table-row")
        time.sleep(1)
        for i in data:
            l = i.text.split(' ')
            province.loc[len(province)] = l

        driver.find_element_by_class_name("ant-pagination-next").click()
        time.sleep(2)
    
    data = driver.find_elements_by_class_name("ant-table-row")
    time.sleep(1)
    for i in data:
        l = i.text.split(' ')
        province.loc[len(province)] = l

    #switch 

    date = driver.find_element_by_class_name("ant-calendar-picker-icon")
    date.click()
    time.sleep(1)
    
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_end[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    
    province2 = pd.DataFrame( columns= header_list)
    
    data = driver.find_elements_by_class_name("ant-table-row")
    while driver.find_element_by_class_name("ant-pagination-next").get_attribute("aria-disabled") == 'false':
        for i in data:
            l = i.text.split(' ')
            province2.loc[len(province2)] = l

        driver.find_element_by_class_name("ant-pagination-next").click()
        time.sleep(2)
        
    data = driver.find_elements_by_class_name("ant-table-row")
    for i in data:
        l = i.text.split(' ')
        province2.loc[len(province2)] = l
    
    province = province[['省市','新增用户','活跃用户','启动次数']]
    province2 = province2[['省市','新增用户','活跃用户','启动次数']]
    province =  pd.merge(province, province2, on='省市',how = 'outer')
    province = province.fillna(0)
    
    province['启动次数_x'] = province['启动次数_x'].apply(get_comma_out)
    province['启动次数_y'] = province['启动次数_y'].apply(get_comma_out)
    province['新增用户_x'] = province['新增用户_x'].apply(get_comma_out)
    province['新增用户_y'] = province['新增用户_y'].apply(get_comma_out)
    province['活跃用户_x'] = province['活跃用户_x'].apply(get_comma_out)
    province['活跃用户_y'] = province['活跃用户_y'].apply(get_comma_out)
    
    province['新增用户_x'] = province['新增用户_x'].astype(int)
    province['新增用户_y'] = province['新增用户_y'].astype(int)
    province['启动次数_x'] = province['启动次数_x'].astype(int)
    province['启动次数_y'] = province['启动次数_y'].astype(int)
    province['活跃用户_x'] = province['活跃用户_x'].astype(int)
    province['活跃用户_y'] = province['活跃用户_y'].astype(int)

    province['新增用户'] = province["新增用户_x"]+province["新增用户_y"]
    province['活跃用户'] = province["活跃用户_x"]+province["活跃用户_y"]
    province['启动次数'] = province["启动次数_x"]+province["启动次数_y"]
    
    province = province[['省市','新增用户','活跃用户','启动次数']]
    
    province.to_csv("C:\\Users\\Ker\\Desktop\\province.csv",encoding = "utf-8_sig")
    
    #switch to country
    button = driver.find_element_by_xpath('//*[@id="scroll-container"]/div/div[2]/div/div[1]/div[2]/div[1]/div/label[2]/span[2]/span')
    button.click()
    time.sleep(1)
    
    date = driver.find_element_by_class_name("ant-calendar-picker-icon")
    date.click()
    time.sleep(1)
    
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    head = driver.find_elements_by_class_name("ant-table-column-title")
    header_list = [i.text for i in head]
    header_list
    
    country = pd.DataFrame( columns= header_list)
    
    data = driver.find_elements_by_class_name("ant-table-row")
    time.sleep(1)
    while driver.find_element_by_class_name("ant-pagination-next").get_attribute("aria-disabled") == 'false':
        for i in data:
            l = i.text.split(' ')
            country.loc[len(country)] = l

        driver.find_element_by_class_name("ant-pagination-next").click()
        time.sleep(2)
        
    data = driver.find_elements_by_class_name("ant-table-row")
    time.sleep(1)
    for i in data:
        l = i.text.split(' ')
        country.loc[len(country)] = l
        
    # switch to date
    
    date = driver.find_element_by_class_name("ant-calendar-picker-icon")
    date.click()
    time.sleep(1)
    
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_end[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    country2 = pd.DataFrame( columns= header_list)
    
    data = driver.find_elements_by_class_name("ant-table-row")
    time.sleep(1)
    while driver.find_element_by_class_name("ant-pagination-next").get_attribute("aria-disabled") == 'false':
        for i in data:
            l = i.text.split(' ')
            country2.loc[len(country2)] = l

        driver.find_element_by_class_name("ant-pagination-next").click()
        time.sleep(2)
    
    data = driver.find_elements_by_class_name("ant-table-row")
    time.sleep(1)
    for i in data:
        l = i.text.split(' ')
        country2.loc[len(country2)] = l
        
    country = country[['国家/地区','新增用户','活跃用户','启动次数']]
    country2 = country2[['国家/地区','新增用户','活跃用户','启动次数']]
    country =  pd.merge(country, country2, on='国家/地区',how = 'outer')
    country = country.fillna(0)
    
    country['启动次数_x'] = country['启动次数_x'].apply(get_comma_out)
    country['启动次数_y'] = country['启动次数_y'].apply(get_comma_out)
    country['新增用户_x'] = country['新增用户_x'].apply(get_comma_out)
    country['新增用户_y'] = country['新增用户_y'].apply(get_comma_out)
    country['活跃用户_x'] = country['活跃用户_x'].apply(get_comma_out)
    country['活跃用户_y'] = country['活跃用户_y'].apply(get_comma_out)
    
    country['新增用户_x'] = country['新增用户_x'].astype(int)
    country['新增用户_y'] = country['新增用户_y'].astype(int)
    country['启动次数_x'] = country['启动次数_x'].astype(int)
    country['启动次数_y'] = country['启动次数_y'].astype(int)
    country['活跃用户_x'] = country['活跃用户_x'].astype(int)
    country['活跃用户_y'] = country['活跃用户_y'].astype(int)

    country['新增用户'] = country["新增用户_x"]+country["新增用户_y"]
    country['活跃用户'] = country["活跃用户_x"]+country["活跃用户_y"]
    country['启动次数'] = country["启动次数_x"]+country["启动次数_y"]

    country = country[['国家/地区','新增用户','活跃用户','启动次数']]
    
    country.to_csv("C:\\Users\\Ker\\Desktop\\country.csv",encoding = "utf-8_sig")

In [9]:
def get_active_user_time(driver, dic_c, date_begin, platform):
    '''
    get active user time in 24 hours for 2 weeks
    '''
    driver.get("https://mobile.umeng.com/platform/{}/reports/active_user".format(platform))
    time.sleep(1)
    
    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    date1 = driver.find_elements_by_class_name("ant-calendar-range-picker-input")
    date1[0].click()
    time.sleep(1)

    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[1])

    time.sleep(1)
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(date_begin[1])
    
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    date1 = driver.find_elements_by_class_name("ant-calendar-range-picker-input")
    date1[0].click()

    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date_begin[0])
    time.sleep(1)

    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[1].send_keys(date_begin[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    button = driver.find_element_by_xpath('//*[@id="scroll-container"]/div/div[2]/div/div[2]/div[1]/div[1]/span/div[2]/div/label[5]/span[2]/span')
    button.click()
    time.sleep(1)
    
    url = "https://mobile.umeng.com/ht/api/v3/app/user/active/trend?relatedId={}&dataSourceId={}".format(platform,platform)
    
    data = {"fromDate":date_begin[0],"toDate":date_begin[1],"version":[],"channel":[],"timeUnit":"hour","view":"activeUser","relatedId":"{}".format(platform),"dataSourceId":"{}".format(platform)}
    
    headers = {
        'accept':"application/json, text/plain, */*",
        "accept-encoding":"gzip, deflate, br",
        "accept-language":"zh-CN,zh;q=0.9",
        "content-length": "192",
        "content-type":"application/json;charset=UTF-8",
        "origin":"https://mobile.umeng.com",
        "referer": "https://mobile.umeng.com/platform/{}/reports/active_user".format(platform),
        "sec-fetch-dest": "empty",
        "sec-fetch-mode" : "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36",
        "x-xsrf-token":"180ef43b-dc03-4746-9e15-b7d856a07d27",
        "x-xsrf-token-haitang": "9286eca8-5ecc-4e77-b073-6bc66f958692"
    }
    
    response =requests.post(url = url,
        json=data,
        cookies=dic_c,
        headers = headers
    )
    
    count = response.json()['data']['items'][0]['data']
    dates = response.json()['data']['dates']
    
    active_user_time1 = pd.DataFrame( columns= ['date','count'])
    active_user_time1.date = dates
    active_user_time1['count'] = count
    
    active_user_time1[['Date','time']] = active_user_time1.date.str.split(expand=True) 
    
    return active_user_time1
    
    

In [10]:
def combine_df(df1,df2):
    df = df1.append(df2)
    df.to_csv("C:\\Users\\Ker\\Desktop\\active_user_time.csv",encoding = "utf-8_sig")

In [11]:
#这边建议将myMailBox改成MailTab

name_dic_ios = {
    'planetTab': '首页tab',
    'explorTab': '想法tab',
    'myMailBox': '信箱tab', 
    'momentTab': '瞬间tab',
    'mineTab': '我的tab',
    'planet_banner': "首页里面的banner",
    'linesDairy': '首页里面的台历',
    'programPlay': '首页里面的节目',
    'rankingList': '首页里面的榜单',
    'planet_hotTopic': "首页里面的k星城邦",
    'filmListDetail': '首页里面的影单',
    'goodFilmSearch':"首页里面的好片库",
    'groupDetail': '城邦',
    'privateMessage': '信箱里面的私信',
    'filmDetail': '影片详情'，
    
    'thought': '一条想法',
    'filmDetail': '电影详情',
    'dairyCollectionList': '手账夹',
    'linesDairyList': '台词日历列表',
    'myViewHistory': '观影历史',
    'planetTab_history': '观影历史',
    'search': '搜索栏',
    'share': '分享',
    'programList': '节目列表',
    'hotFilmList_banner': '热门影单banner',
    'hotFilmList_normal': '热门影单普通',
    'mineTab_myFilmList': '我的页面我的影单区域',
    'mineTab_myFilmListCollection': '我的页面我收藏的影单区域',
    'mineTab_myFilmListShare': '我的页面共享的影单区域',
    'mineTab_recommend': '我的页面推荐影单',
    'myFilmList': '我的影单页面',
    'personalHomepage': '个人主页',
    'planetTab_recommendForYou': 'k星球推荐',
    'myGroups': '我的页面进我的城邦',
    'explorTabTopic': '想法话题',
    'thoughtsTag': '想法tag',
    "explorTabGroups_popular":"unknown"
}

In [12]:
input_dic_ios = {
    'planetTab': '首页tab',
    'explorTab': '想法tab',
    'myMailBox': '信箱tab', 
    'momentTab': '瞬间tab',
    'mineTab': '我的tab',
    'planet_banner': "首页里面的banner",
    'linesDairy': '首页里面的台历',
    'programPlay': '首页里面的节目',
    'rankingList': '首页里面的榜单',
    'planet_hotTopic': "首页里面的k星城邦",
    'filmListDetail': '首页里面的影单',
    'goodFilmSearch':"首页里面的好片库",
    'groupDetail': '城邦',
    'privateMessage': '信箱里面的私信'
}

In [13]:
input_dic_android = {
    'com.dianyingker.movie.module.common.MainActivity': '主页',
    'com.dianyingker.movie.module.home.kstar.KStarOriginalDetailActivity': '节目',
    'com.dianyingker.movie.module.home.detail.MovieDetailActivity': '影片详情',
    'com.dianyingker.movie.module.account.message.ChatWithOfficialActivity': 'k小白私信',
    'com.dianyingker.movie.module.account.person.HomePageActivity': '个人主页',
    'com.dianyingker.movie.module.user.mine.MovieGroupDetailActivity': '影单'
}

In [14]:
def name_match_ios(s):
    return input_dic_ios[s]

In [15]:
def name_match_Android(s):
    return input_dic_android[s]

In [16]:
def remove_parentheses(s):
    return s.split("(")[0]

In [17]:
def remove_parentheses2(s):
    return s.split("秒(")[0]

In [18]:
def remove_percent_mark(s):
    return s[:-1]

In [19]:
def get_path_overall_data(driver, dic_c, date3, platform):
    '''
    获取path的整体数据
    '''
    driver.get("https://mobile.umeng.com/platform/{}/function/path".format(platform))
    time.sleep(1)

    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    date = driver.find_elements_by_class_name("um-ui-single-range-picker-text")
    date[0].click()
    
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(Keys.CONTROL + "a") 
    driver.find_elements_by_class_name("ant-calendar-input")[0].send_keys(date3[1])
    time.sleep(1)
    
    mouse_action = ActionChains(driver)
    mouse_action.move_by_offset(xoffset = 0 , yoffset = 55).click().perform()
    time.sleep(1)
    
    head = driver.find_elements_by_class_name("ant-table-column-title")
    header_list = [i.text for i in head]
    
    path_overall = pd.DataFrame( columns= header_list[:-3])
    
    data = driver.find_elements_by_class_name("ant-table-row")
    
    for i in data:
        a = i.text
        l = a.split(" ")
        l1 = l[:3]
        l2 = l[-1]
        a = "".join(l[3:5])
        l1.append(a)
        l1.append(l2)
        path_overall.loc[len(path_overall)] = l1
        
    if platform == "5cbd86bd570df3cf0a0009ee":
        #print("this is for ios")
        path_overall = path_overall[path_overall.描述.isin(list(input_dic_ios.keys()))]
        path_overall['描述'] = path_overall.描述.apply(name_match_ios)
    else:
        path_overall = path_overall[path_overall.描述.isin(list(input_dic_android.keys()))]
        #print("this is for Android")
        path_overall['描述'] = path_overall.描述.apply(name_match_Android)
        
     
    path_overall['访问次数'] = path_overall["访问次数(占比)"].apply(remove_parentheses)
    path_overall['平均访问时长'] = path_overall["平均访问时长(占比)"].apply(remove_parentheses2)
    path_overall['跳出率'] = path_overall["跳出率"].apply(remove_percent_mark)
    path_overall = path_overall[['页面(Activity/Fragment)','描述','访问次数','平均访问时长','跳出率']]
         
    path_overall.to_csv("C:\\Users\\Ker\\Desktop\\path_{}.csv".format(date3[1]),encoding = "utf-8_sig")
        
        
       
    return 
   

In [20]:
def get_path_detail(driver, dic_c, date1, date2, platform):
    '''
    get all the detail path information
    '''
    
    driver.get("https://mobile.umeng.com/platform/{}/function/events/dashboard".format(platform))
    time.sleep(1)

    for i in dic_c:
        driver.add_cookie({'name':i,'value': dic_c[i]})
    driver.refresh()
    time.sleep(2)
    
    return
    
    
        

In [ ]:
def get_all_data(s,date1,date2,platform):
    '''
    get all the data includes:
        1. total trend
        2. new user stay
        3. active user stay
        4. device
        5. country
        6. location
        7. active time in hour
    '''
    driver = open_window()
    #填写你网站的cookie
    #填写你想获取的两周时间段

    dic_c = get_cookie_dic(s)

    
    #下载两周的总体趋势（total_trend.csv）
    get_total_trend(driver, dic_c , date1, date2, platform)

    #下载两周(新用户+活跃用户）留存 (new_use_stay.csv, active_user_stay.csv)
    get_user_stay(driver, dic_c, date1, date2, platform)

    #下载两周device的数据信息 (device.csv
    get_device(driver,dic_c, date1, date2, platform)

    #下载两周省份和国家的数据信息（province.csv, country.csv)
    get_location(driver, dic_c, date1, date2, platform)

    #下载两周的活跃时间
    combine_df(get_active_user_time(driver, dic_c, date1, platform),get_active_user_time(driver, dic_c, date2, platform))
    
    get_path_overall_data(driver, dic_c, date1, platform)
    get_path_overall_data(driver, dic_c, date2, platform)
    
    #get_path_detail(driver, dic_c, date1, date2, platform)
    
    

## Back-end Data

### 目前后台的数据缺少： 整体新增（这个需要手动）

In [7]:
def getBetweenDay(begin_date,end_date):
    '''
    get date between begin_date and end_date (included)
    '''
    date_list = []
    begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += datetime.timedelta(days=1)
    return date_list

In [23]:
def get_cb_data(cookie, date1, date2, dic_h):
    '''
    get cb data for two weeks
    '''
    
    df = pd.DataFrame( columns= ['#', '城邦ID', '邦主头像', '邦主昵称', '创建人头像', '创建人昵称', '创建时间', '城邦类型', '城邦', '阅读量', '想法量', '订阅量', '是否精选', '是否上首页', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/topic/index/{}?time_1={}+&time_2={}+&is_selected=0&is_go_home=0&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/topic/index/{}?time_1={}+&time_2={}+&is_selected=0&is_go_home=0&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    tr_list = soup.find("table",id="checkAll").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
            
    df.to_csv("C:\\Users\\Ker\\Desktop\\cb.csv",encoding = "utf-8_sig")
    return 



In [24]:
def get_yd_data(cookie, date1, date2, dic_h):
    '''
    get cb data for two weeks
    '''
        
    df = pd.DataFrame( columns= ['#', 'ID', '头像', '昵称', '创建时间', '名称', '描述', '横向封面', '共享量', '收藏量', '查看量', '影片量', '电影', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/movieGroup/index/{}?time_1={}&time_2={}&min_num=&max_num=&is_selected=0&is_go_home=0&is_hcover=0&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="datalist").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/movieGroup/index/{}?time_1={}&time_2={}&min_num=&max_num=&is_selected=0&is_go_home=0&is_hcover=0&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="datalist").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
    
    df.to_csv("C:\\Users\\Ker\\Desktop\\yd.csv",encoding = "utf-8_sig")
    return 

In [25]:
def get_jspl_data(cookie, date1, date2, dic_h):
    '''
    获取解说评论数据
    '''
    df = pd.DataFrame( columns= ['#', '评论ID', '粉丝头像', '粉丝', '评论时间', '视频', '节目编号', '评论', '点赞数', '置顶', '删除', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/comment/index/{}?time_1={}&time_2={}&is_top=-1&is_deleted=-1&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/comment/index/{}?time_1={}&time_2={}&is_top=-1&is_deleted=-1&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="checkAll").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
    
    df.to_csv("C:\\Users\\Ker\\Desktop\\jspl.csv",encoding = "utf-8_sig")
    return 
    

In [26]:
def get_xflb_data(cookie, date1, date2, dic_h):
    '''
    获取想法列表数据
    '''
    df = pd.DataFrame( columns= ['#', '想法ID', '头像', '昵称', '想法类型', '内容', '权限设置', '城邦', '评论量', '点赞量', '发布时间', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/dynamic/index/{}?time_1={}&time_2={}&is_deleted=0&is_selected=0&is_go_home=0&keyword=&topic=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="datalist").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/dynamic/index/{}?time_1={}&time_2={}&is_deleted=0&is_selected=0&is_go_home=0&keyword=&topic=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="datalist").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
    
    df.to_csv("C:\\Users\\Ker\\Desktop\\xflb.csv",encoding = "utf-8_sig")
    return 

In [24]:
def get_tcdm_data(cookie, date1,date2, dic_h):
    '''
    获取台词日历弹幕
    '''
        
    df = pd.DataFrame( columns= ['#', '日历', '粉丝头像', '粉丝昵称', '日历日期', '台词', '弹幕内容', '弹幕发送时间', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/barrage/index/{}?keyword=&day=&time_1={}&time_2={}&dosubmit=%E6%90%9C%E7%B4%A2'
        .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/barrage/index/{}?keyword=&day=&time_1={}&time_2={}&dosubmit=%E6%90%9C%E7%B4%A2'
            .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="checkAll").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
    
            
    df.to_csv("C:\\Users\\Ker\\Desktop\\tcdm.csv",encoding = "utf-8_sig")
    return

In [28]:
def get_jsdm_data(cookie, date1, date2, dic_h):
    '''
    获取解说弹幕
    '''
    df = pd.DataFrame( columns= ['#', '粉丝头像', '粉丝', '弹幕发送时间', '视频名', '弹幕内容', '弹幕颜色', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/movieBarrage/index/{}?keyword=&time_1={}&time_2={}&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/movieBarrage/index/{}?keyword=&time_1={}&time_2={}&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="checkAll").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
    
    df.to_csv("C:\\Users\\Ker\\Desktop\\jsdm.csv",encoding = "utf-8_sig")
    
    return 
    

In [29]:
def get_yppl_data(cookie, date1, date2, dic_h):
    '''
    获取影片评论弹幕
    '''
    df = pd.DataFrame( columns= ['#', '评论ID', '粉丝头像', '粉丝', '评论时间', '视频', '打分', '评论', '点赞数', '置顶', '删除', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/comment/film/{}?time_1={}+&time_2={}+&is_top=-1&is_deleted=-1&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        tr_list2 = tr_list[1::3]
        for i in tr_list2:
            cols = i.find_all('td')
            rows = [x.text.strip() for x in cols]
            rows2 = rows[:7]+(rows[-5:])
            df.loc[len(df)] = rows2
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/comment/film/{}?time_1={}+&time_2={}+&is_top=-1&is_deleted=-1&keyword=&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="checkAll").find_all('tr')
    tr_list2 = tr_list[1::3]
    for i in tr_list2:
        cols = i.find_all('td')
        rows = [x.text.strip() for x in cols]
        rows2 = rows[:7]+(rows[-5:])
        df.loc[len(df)] = rows2
    
    df.to_csv("C:\\Users\\Ker\\Desktop\\yppl.csv",encoding = "utf-8_sig")
    return 

In [30]:
def get_xfpl_data(cookie, date1, date2, dic_h):
    '''
    获取想法评论数据
    '''
    df = pd.DataFrame( columns= ['#', '评论ID', '粉丝头像', '粉丝', '评论时间', '想法', '评论', '点赞数', '删除', '回复', '操作'])
    count = 1
    r = requests.get('https://www.dianyingker.com/adminpanel/comment/dynamic/{}?keyword=&time_1={}&time_2={}&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count, date1[0],date2[1]), headers = dic_h , cookies = cookie)
    soup = BeautifulSoup(r.text, 'lxml')

    print("download page 1 ...")
    while soup.find("a", class_ = "nextpage") != None:
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
        count += 1
        print("download page {} ...".format(count))
        r = requests.get('https://www.dianyingker.com/adminpanel/comment/dynamic/{}?keyword=&time_1={}&time_2={}&dosubmit=%E6%90%9C%E7%B4%A2'
                              .format(count,date1[0],date2[1] ), headers = dic_h , cookies = cookie)
        soup = BeautifulSoup(r.text, 'lxml')
        
    
    tr_list = soup.find("table",id="checkAll").find_all('tr')
    for j in range(1,len(tr_list)):
        row = [x.text.strip() for x in tr_list[j].find_all('td')]
        df.loc[len(df)] = row
    
    df.to_csv("C:\\Users\\Ker\\Desktop\\xfpl.csv",encoding = "utf-8_sig")
    return 

In [31]:
def get_new_user_data(cookie, date1, date2, dic_h ):
    '''
    获取新用户数据
    '''
    return 

In [32]:
def get_all_backend_data(cookie, date1, date2, dic_h):
    '''
    get all back end data together
    '''
    
    # 城邦数据
    print("start with 城邦 data ...")
    get_cb_data(cookie, date1, date2, dic_h)
    print("end with 城邦 data ...")
    print("-------------------------------")
    
    # 影单数据
    print("start with 影单 data ...")
    get_yd_data(cookie, date1, date2, dic_h)
    print("end with 影单 data ...")
    print("-------------------------------")
    
    # 解说评论
    print("start with 解说评论 data ...")
    get_jspl_data(cookie, date1, date2, dic_h)
    print("end with 解说评论 data ...")
    print("-------------------------------")
    
    # 想法列表
    print("start with 想法列表 data ...")
    get_xflb_data(cookie, date1, date2, dic_h)
    print("end with 想法列表 data ...")
    print("-------------------------------")
    
    # 台词弹幕
    print("start with 台词弹幕 data ...")
    get_tcdm_data(cookie, date1,date2, dic_h)
    print("end with 台词弹幕 data ...")
    print("-------------------------------")
    
    # 解说弹幕
    print("start with 解说弹幕 data ...")
    get_jsdm_data(cookie, date1, date2, dic_h)
    print("end with 解说弹幕 data ...")
    print("-------------------------------")
    
    # 影片评论
    print("start with 影片评论 data ...")
    get_yppl_data(cookie, date1, date2, dic_h)
    print("end with 影片评论 data ...")
    print("-------------------------------")
    
    # 想法评论
    print("start with 想法评论 data ...")
    get_xfpl_data(cookie, date1, date2, dic_h)
    print("end with 想法评论 data ...")
    print("-------------------------------")
    
    
    # 新增用户
    print("start with 新增用户 data ...")
    get_new_user_data(cookie, date1, date2, dic_h)
    print("end with 想法评论 data ...")
    print("-------------------------------")

In [33]:
def get_new_user_info(startpage, endpage):
    '''
    get new user register time information from backend data for 14 days (from 2020-08-02 to 2020-08-15)
    n is the page 
    '''
    df = pd.DataFrame( columns= ['#', '用户ID', '用户头像', '昵称', '邀请码', '手机号码', '是否黑名单', '是否内部员工', '是否运营号', '是否终身会员', '注册APP时间', '最近一条消息', '操作'])
    for i in range(startpage,endpage+1):
        print("start download {} page ...".format(i))
        r = requests.get("https://www.dianyingker.com/adminpanel/fans/index/{}?order=register_time_in_app&dir=desc".format(i), headers = headers , cookies = s)
        soup = BeautifulSoup(r.text, 'lxml')
        tr_list = soup.find("table",id="checkAll").find_all('tr')
        for j in range(1,len(tr_list)):
            row = [x.text.strip() for x in tr_list[j].find_all('td')]
            df.loc[len(df)] = row
    return df

# Main

In [5]:
# which dates you wanna scrape the data
date1 = ['2020-08-16','2020-08-22']
date2 = ['2020-08-23','2020-08-29']

# s1 -> ios cookie 
s1 = "UM_distinctid=173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465; dplus_cross_id=173e12506d628c-06078ac4c8da6f-3323767-1fa400-173e12506d759e; dplus_finger_print=2822180395; cna=Omy6F8MK9Q0CARsRIOYQfmm8; uc_session_id=973cf1ce-0641-40d4-984c-e6e9e1b4cb88; umplus_uc_loginid=kdylm; cn_1271885898_dplus=1%5B%7B%22userid%22%3A%22kdylm%22%2C%22AppKey%22%3A%225cbd86bd570df3cf0a0009ee%22%7D%2C0%2C1597977420%2C0%2C1597977420%2Cnull%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597285046%22%2C%22https%3A%2F%2Fworkbench.umeng.com%2F%22%2C%22workbench.umeng.com%22%5D; umplus_uc_token=1SG7yAOH25SI32wcCDFEHpg_62aad45450274e1d92ae5b255e088713; Hm_lvt_289016bc8d714b0144dc729f1f2ddc0d=1598508153,1598508271,1598581271,1598923938; XSRF-TOKEN=180ef43b-dc03-4746-9e15-b7d856a07d27; XSRF-TOKEN-HAITANG=9286eca8-5ecc-4e77-b073-6bc66f958692; Hm_lpvt_289016bc8d714b0144dc729f1f2ddc0d=1598924867; cn_1258498910_dplus=1%5B%7B%22utm_source%22%3A%22n_uapp_top_pc_c%22%2C%22userid%22%3A%22kdylm%22%7D%2C0%2C1598924868%2C0%2C1598924868%2C%22%24direct%22%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597207562%22%2C%22https%3A%2F%2Fwww.google.com%2F%22%2C%22www.google.com%22%5D; EGG_SESS=_GOCmDZcFN18IbC7Ny32_jE4YaQ_aYwZxhPACRjThr8yPAC3cAv-zhtTdnQpsB0Dgf5MCX_hizRFzqXCI0-i1W6kEvgqNBcyquRYqxn3KsYH4kK-XGNXZ5xJb8GVZKL4QDbn6FvQ9YALpDtZrtHjTPFgEETMFJ52MuXnUAgcrhZJgss19PD4eQomFH0iMpMxJ-4N3n-fiovISjKRg4u6sjOCOGVALRwMWhytWABECUP7rd_5q3QgQX5yCdD3mW4IkfL8Sv4Yahd8ivC2ga8HQm5ezyk1MZ7OY6FtSJR3qpY2mOSxbDnt5pOFS9Q6c2BoFdoE8iRhsQHzf8feqLp5jI1Zzgpp3nJIelCLYNQ1XN94Iogghm_61-WQ8yQlX_mANfNz28p_E5GLk11jxjMre79b28qd6gr1vILQ95SXH5C1Rw5W3-9q5PnK1lNKenUpQM4KcShEJgTmRLepB7Excp0TZHim2mdSdB7rhUp0SV3xW3M3-W3Oebi_ygoGTBwB6xDSneXT2oKr2IVa1IkHxyuVNv6CTNI1NMSEcT38X_72WL5otovJYUgiWoQzR0Q0aPgYXKEFFM8wdkUA-dT3MXXamCeXTz6P2FTqMwMdCdHhx0-xE84rehl00Ds8y3GdEtuncuInQIGIoObSvh6Yzq2HsIKYvl2l6HmNepmbA5gAJ7-X12d_-zMuTteubqz43sQNSo30cWAOCKRLdh0DIewci8uR5u4HDIqKU5qx9gk=; cn_1273967994_dplus=1%5B%7B%7D%2Cnull%2Cnull%2Cnull%2Cnull%2C%22%24direct%22%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597209918%22%2C%22https%3A%2F%2Fworkbench.umeng.com%2F%3Fspm%3Da213m0.13887608.0.0.3cb275efYgGziK%22%2C%22workbench.umeng.com%22%5D; CNZZDATA1259864772=449313210-1597209918-https%253A%252F%252Fworkbench.umeng.com%252F%7C1598933903; isg=BGVlStPVVikPt7KLqcGHFfb1dCGfohk0vt7zSWdPURynfpXwL_TBBAJfDOII_jHs; cn_1259864772_dplus=1%5B%7B%22Uapp_appkey%22%3A%225cbd86bd570df3cf0a0009ee%22%2C%22Uapp_platform%22%3A%22iphone%22%2C%22UserID%22%3A%22kdylm%22%7D%2C0%2C1598938296%2C0%2C1598938296%2C%22%24direct%22%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597209918%22%2C%22https%3A%2F%2Fworkbench.umeng.com%2F%3Fspm%3Da213m0.13887608.0.0.3cb275efYgGziK%22%2C%22workbench.umeng.com%22%5D"
sios = "5cbd86bd570df3cf0a0009ee"

# s2 -> Android cookie
s2 = "UM_distinctid=173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465; dplus_cross_id=173e12506d628c-06078ac4c8da6f-3323767-1fa400-173e12506d759e; dplus_finger_print=2822180395; cna=Omy6F8MK9Q0CARsRIOYQfmm8; uc_session_id=973cf1ce-0641-40d4-984c-e6e9e1b4cb88; umplus_uc_loginid=kdylm; cn_1271885898_dplus=1%5B%7B%22userid%22%3A%22kdylm%22%2C%22AppKey%22%3A%225cbd86bd570df3cf0a0009ee%22%7D%2C0%2C1597977420%2C0%2C1597977420%2Cnull%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597285046%22%2C%22https%3A%2F%2Fworkbench.umeng.com%2F%22%2C%22workbench.umeng.com%22%5D; umplus_uc_token=1SG7yAOH25SI32wcCDFEHpg_62aad45450274e1d92ae5b255e088713; EGG_SESS=_GOCmDZcFN18IbC7Ny32_jE4YaQ_aYwZxhPACRjThr8yPAC3cAv-zhtTdnQpsB0Dgf5MCX_hizRFzqXCI0-i1W6kEvgqNBcyquRYqxn3KsYH4kK-XGNXZ5xJb8GVZKL4QDbn6FvQ9YALpDtZrtHjTPFgEETMFJ52MuXnUAgcrhZJgss19PD4eQomFH0iMpMxJ-4N3n-fiovISjKRg4u6sjOCOGVALRwMWhytWABECUP7rd_5q3QgQX5yCdD3mW4IkfL8Sv4Yahd8ivC2ga8HQm5ezyk1MZ7OY6FtSJR3qpY2mOSxbDnt5pOFS9Q6c2BoFdoE8iRhsQHzf8feqLp5jI1Zzgpp3nJIelCLYNQ1XN94Iogghm_61-WQ8yQlX_mANfNz28p_E5GLk11jxjMre79b28qd6gr1vILQ95SXH5C1Rw5W3-9q5PnK1lNKenUpQM4KcShEJgTmRLepB7Excp0TZHim2mdSdB7rhUp0SV3xW3M3-W3Oebi_ygoGTBwB6xDSneXT2oKr2IVa1IkHxyuVNv6CTNI1NMSEcT38X_72WL5otovJYUgiWoQzR0Q0aPgYXKEFFM8wdkUA-dT3MXXamCeXTz6P2FTqMwMdCdHhx0-xE84rehl00Ds8y3GdEtuncuInQIGIoObSvh6Yzq2HsIKYvl2l6HmNepmbA5jRdg06r4E9C5Oj85xnixnaOEXdnpi4gvr2JL6AHzxP0Y2KbMHp_dA7Pyn1bYJ3WEo=; Hm_lvt_289016bc8d714b0144dc729f1f2ddc0d=1598508153,1598508271,1598581271,1598923938; CNZZDATA1259864772=449313210-1597209918-https%253A%252F%252Fworkbench.umeng.com%252F%7C1598920258; XSRF-TOKEN=180ef43b-dc03-4746-9e15-b7d856a07d27; XSRF-TOKEN-HAITANG=9286eca8-5ecc-4e77-b073-6bc66f958692; Hm_lpvt_289016bc8d714b0144dc729f1f2ddc0d=1598924867; cn_1258498910_dplus=1%5B%7B%22utm_source%22%3A%22n_uapp_top_pc_c%22%2C%22userid%22%3A%22kdylm%22%7D%2C0%2C1598924868%2C0%2C1598924868%2C%22%24direct%22%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597207562%22%2C%22https%3A%2F%2Fwww.google.com%2F%22%2C%22www.google.com%22%5D; cn_1259864772_dplus=1%5B%7B%22Uapp_appkey%22%3A%225cbd86520cafb282c9000f14%22%2C%22Uapp_platform%22%3A%22android%22%2C%22UserID%22%3A%22kdylm%22%7D%2C0%2C1598925419%2C0%2C1598925419%2C%22%24direct%22%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597209918%22%2C%22https%3A%2F%2Fworkbench.umeng.com%2F%3Fspm%3Da213m0.13887608.0.0.3cb275efYgGziK%22%2C%22workbench.umeng.com%22%5D; cn_1273967994_dplus=1%5B%7B%7D%2Cnull%2Cnull%2Cnull%2Cnull%2C%22%24direct%22%2C%22173e12506d2565-0760ced0230ae4-3323767-1fa400-173e12506d3465%22%2C%221597209918%22%2C%22https%3A%2F%2Fworkbench.umeng.com%2F%3Fspm%3Da213m0.13887608.0.0.3cb275efYgGziK%22%2C%22workbench.umeng.com%22%5D; isg=BJGR0mb_qu21TMaHne3rUQoRoJ0r_gVwN2whYHMmsdh3GrBsu0x5QNd8vO78Ep2o"
sAndroid = "5cbd86520cafb282c9000f14"

#----------------------------------------------------------------------------------------------------------------------------
#s -> backend cookie 

s = {"autocodeigniter_com":"1cakpn78ptacqs71v1ap3og6jqbki2fa"}
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"}


In [ ]:
# 后台数据
get_all_backend_data(s, date1, date2, headers )

In [137]:
#get ios data:
get_all_data(s1,date1,date2,sios)

### Don't start IOS and Android download together because they have same .csv name
### ONLY start Android after locating IOS data in the corresponding folder

In [146]:
#get Andriod data:
get_all_data(s2,date1,date2,sAndroid)